In [ ]:
import numpy as np
import cv2
import torch
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from models.yolo import Model
import math
import time

# Add the custom class to the safe globals list
torch.serialization.add_safe_globals([Model])

# Initialize device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load YOLOv7-pose model
weights = torch.load('yolov7-w6-pose.pt', map_location=device, weights_only=False)
model = weights['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)

# Constants from the journal
ALPHA = 0.5  # Adjustment factor for height threshold
SPEED_THRESHOLD = 0.5  # Vertical speed threshold for fall detection (pixels/frame)
ANGLE_THRESHOLD = 45  # Degrees threshold between torso and legs
TARGET_FPS = 25  # Target frames per second for processing

# Previous frame data for speed calculation
prev_shoulder_y = None
frame_count = 0

def calculate_length_factor(shoulder, torso):
    return np.sqrt((shoulder[0] - torso[0])*2 + (shoulder[1] - torso[1])*2)

def calculate_angle(a, b, c):
    """Calculate angle between points a, b, c in degrees"""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

def detect_fall(keypoints, threshold=0.5):
    global prev_shoulder_y, prev_frame_time, fall_start_time
    
    # Keypoint indices
    NOSE = 0
    LEFT_SHOULDER, RIGHT_SHOULDER = 5, 6
    LEFT_HIP, RIGHT_HIP = 11, 12
    LEFT_ANKLE, RIGHT_ANKLE = 15, 16
    LEFT_KNEE, RIGHT_KNEE = 13, 14

    # Initialize default return values
    is_fall = False
    current_state = "normal"
    conditions = []
    
    try:
        # Extract keypoints with confidence check
        kp = {}
        for name, idx in [('nose', NOSE), ('left_shoulder', LEFT_SHOULDER),
                         ('right_shoulder', RIGHT_SHOULDER), ('left_hip', LEFT_HIP),
                         ('right_hip', RIGHT_HIP), ('left_knee', LEFT_KNEE),
                         ('right_knee', RIGHT_KNEE), ('left_ankle', LEFT_ANKLE),
                         ('right_ankle', RIGHT_ANKLE)]:
            kp[name] = keypoints[idx*3:(idx+1)*3]
            if kp[name][2] < threshold:
                return False, "low_confidence", ["Low confidence in keypoints"]

        # 1. Calculate critical distances
        torso_length = math.sqrt((kp['left_shoulder'][0]-kp['left_hip'][0])**2 + 
                              (kp['left_shoulder'][1]-kp['left_hip'][1])**2)
        shoulder_height = (kp['left_shoulder'][1] + kp['right_shoulder'][1]) / 2
        feet_height = (kp['left_ankle'][1] + kp['right_ankle'][1]) / 2
        head_height = kp['nose'][1]

        # 2. Normalized height ratios
        head_to_feet = abs(head_height - feet_height)
        shoulder_to_feet = abs(shoulder_height - feet_height)
        normalized_ratio = shoulder_to_feet / (head_to_feet + 1e-5)

        # 3. Vertical speed calculation
        current_time = time.time()
        vertical_speed = 0
        if prev_shoulder_y is not None and prev_frame_time is not None:
            time_elapsed = current_time - prev_frame_time
            if time_elapsed > 0:
                vertical_speed = (shoulder_height - prev_shoulder_y) / time_elapsed
        
        # 4. Body orientation
        body_width = abs(kp['left_shoulder'][0] - kp['right_shoulder'][0])
        body_height = head_to_feet
        orientation_ratio = body_width / (body_height + 1e-5)

        # 5. Leg angles
        def calculate_angle(a, b, c):
            ba = np.array(a[:2]) - np.array(b[:2])
            bc = np.array(c[:2]) - np.array(b[:2])
            cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
            return np.degrees(np.arccos(np.clip(cosine_angle, -1, 1)))

        left_leg_angle = calculate_angle(kp['left_hip'], kp['left_knee'], kp['left_ankle'])
        right_leg_angle = calculate_angle(kp['right_hip'], kp['right_knee'], kp['right_ankle'])
        min_leg_angle = min(left_leg_angle, right_leg_angle)

        # Fall Detection Conditions
        conditions = []
        
        # Shoulders near feet
        shoulder_foot_threshold = 0.8 * torso_length
        if shoulder_height > feet_height - shoulder_foot_threshold:
            conditions.append("shoulders_near_feet")
        
        # Rapid downward movement
        if vertical_speed > SPEED_THRESHOLD:
            conditions.append(f"rapid_downward_{vertical_speed:.1f}px/s")
        
        # Body orientation
        if orientation_ratio > 1.2:
            conditions.append("horizontal_posture")
        
        # Leg collapse
        if min_leg_angle < ANGLE_THRESHOLD:
            conditions.append(f"legs_collapsed_{min_leg_angle:.0f}deg")

        # Fall state determination
        if "shoulders_near_feet" in conditions:
            if any("rapid_downward" in cond for cond in conditions):
                current_state = "falling"
                fall_start_time = current_time
            elif "horizontal_posture" in conditions and current_time - fall_start_time < 1.0:
                current_state = "fallen"
        
        # Final decision
        if current_state == "fallen":
            is_fall = True
        elif len(conditions) >= 2 and "shoulders_near_feet" in conditions:
            is_fall = True
        
        # Update tracking variables
        prev_shoulder_y = shoulder_height
        prev_frame_time = current_time
        
    except Exception as e:
        conditions = [f"Error: {str(e)}"]
        return False, "error", conditions
    
    return is_fall, current_state, conditions

# Path to video file
video_path = "C:\\Users\\LENOVO\\Documents\\A Skripsi\\datasets\\FallDataset\\Dataset\\Coffee_room_01\\Videos\\video (4).avi"
cap = cv2.VideoCapture(video_path)

# Get video properties
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Calculate skip frames to achieve 25 FPS
if original_fps > 0:
    skip_frames = max(1, int(round(original_fps / TARGET_FPS)))
else:
    skip_frames = 1  # Default if FPS info not available

print(f"Original FPS: {original_fps}, Processing at {TARGET_FPS} FPS, skipping {skip_frames-1} frames between processed frames")

# Create video writer to maintain 25 FPS output
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, TARGET_FPS, (640, 640))

frame_counter = 0
start_time = time.time()
processed_frames = 0
last_frame_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_counter += 1
    
    # Skip frames to achieve target FPS
    if frame_counter % skip_frames != 0:
        continue
    
    # Calculate time to sleep to maintain 25 FPS
    current_time = time.time()
    elapsed = current_time - last_frame_time
    sleep_time = max(0, (1.0/TARGET_FPS) - elapsed)
    time.sleep(sleep_time)
    last_frame_time = time.time()
    
    processed_frames += 1
    frame_count += 1
    
    image = letterbox(frame, 640, stride=64, auto=True)[0]
    image_ = image.copy()
    image = transforms.ToTensor()(image)
    image = torch.tensor(np.array([image.numpy()]))

    if torch.cuda.is_available():
        image = image.half().to(device)

    with torch.no_grad():
        output, _ = model(image)
        output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
        output = output_to_keypoint(output)

    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)

    for idx in range(output.shape[0]):
        keypoints = output[idx, 7:].T
        plot_skeleton_kpts(nimg, keypoints, 3)
    
        # Updated to handle 3 return values
        fall_detected, state, conditions = detect_fall(keypoints)
        
        if fall_detected:
            cv2.putText(nimg, f"FALL: {state}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            # Display all triggered conditions
            for i, condition in enumerate(conditions):
                cv2.putText(nimg, condition, (50, 90 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            cv2.putText(nimg, f"State: {state}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            if conditions:  # Show conditions even when no fall
                cv2.putText(nimg, f"Conditions: {', '.join(conditions)}", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    # Calculate and display actual processing FPS
    elapsed_time = time.time() - start_time
    current_fps = processed_frames / elapsed_time if elapsed_time > 0 else 0
    cv2.putText(nimg, f"Processing FPS: {current_fps:.1f}", (50, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Write frame to output video
    out.write(nimg)
    
    cv2.imshow("Fall Detection", nimg)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# Calculate final processing statistics
total_time = time.time() - start_time
actual_fps = processed_frames / total_time if total_time > 0 else 0
print(f"Processing complete. Actual FPS: {actual_fps:.2f}")

cap.release()
out.release()
cv2.destroyAllWindows()

mending lanjut coding diatas ini bang, belum 100% rumusnya sama ama di jurnal tapi